# 1. LOADING

In [ ]:
# import packages
import pandas as pd
import graphlab as gl
from sklearn import metrics
from scipy import stats

In [ ]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

In [ ]:
# print variable names
#data.columns

# 2. DATA PREPARATION

In [ ]:
# converting factors to strings
data["user_id"]   = data.user_id.astype(str)
data["media_id"]  = data.media_id.astype(str)
data["album_id"]  = data.album_id.astype(str)
data["artist_id"] = data.artist_id.astype(str)
data["genre_id"]  = data.genre_id.astype(str)

In [ ]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

In [ ]:
# data sets with user-media interactions
tr_listen = gl.SFrame(tr[["media_id", "user_id", "is_listened"]])
ts_listen = gl.SFrame(ts[["media_id", "user_id", "is_listened"]])
kn_listen = gl.SFrame(kn[["media_id", "user_id", "is_listened"]])
un_listen = gl.SFrame(un[["media_id", "user_id", "is_listened"]])

In [ ]:
# clearing the memory
data = "Null"
tr   = "Null"
kn   = "Null"

# 3. MODELING

## 3.1. TRAINING-VALIDATION

In [ ]:
# model parameters
k = 20000
sim_metric = "cosine"

In [ ]:
# training the model
model = gl.recommender.item_similarity_recommender.create(tr_listen, 
                                                          only_top_k = k,
                                                          similarity_type = sim_metric, 
                                                          user_id = "user_id", 
                                                          item_id = "media_id", 
                                                          target  = "is_listened")

In [ ]:
# forecasting
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict(ts_listen)
pred.head(3)

In [ ]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred["is_listened"])

In [ ]:
# saving prediction vector
pred.to_csv(path + "pred_valid/similarity_scores_flow_20000k.csv", index = False)

In [ ]:
# clearing the memory
tr_listen = "Null"
ts = "Null"

## 3.2. KNOWN-UNKNOWN

In [ ]:
# training the model
model = gl.recommender.item_similarity_recommender.create(kn_listen, 
                                                          only_top_k = k,
                                                          similarity_type = sim_metric, 
                                                          user_id = "user_id", 
                                                          item_id = "media_id", 
                                                          target  = "is_listened")

In [ ]:
# forecasting
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict(un_listen)
pred = pred.sort_values("sample_id")
pred.head(3)

In [ ]:
# saving prediction vector
pred.to_csv(path + "pred_unknown/similarity_scores_flow_20000k.csv", index = False)